<a href="https://colab.research.google.com/github/rochadelon/Uece-C-Jovem/blob/Ci%C3%AAncia-de-Dados-Intermedi%C3%A1rio/Oficina_3_Plano_de_prepara%C3%A7%C3%A3o_de_dados_para_modelo_preditivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Alan Delon Sousa Rocha
*   C-Jovem - Uece
*   Ciência de Dados - Intermediário

# Oficina 3 - Plano de preparação de dados para modelo preditivo

**Desafio:**

Você é um analista de dados em uma grande empresa de varejo on-line que acaba de concluir uma campanha de marketing direcionada. A empresa coletou diversos dados sobre os clientes, incluindo informações demográficas, comportamento de compra e respostas à campanha. Agora, sua tarefa é preparar esse conjunto de dados para construir um modelo preditivo que ajudará a empresa a segmentar melhor seus clientes em campanhas futuras.

O conjunto de dados contém algumas inconsistências, valores faltantes e variáveis categóricas que precisam ser tratadas para garantir que o modelo preditivo seja preciso e eficaz, conforme apresentado na tabela abaixo:

In [ ]:
from google.colab import files
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

df = pd.DataFrame ({
    "Cliente ID": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "Idade": [25, 34, 28, 45, 39, None, 50, 23, 29, 40],
    "Gênero": ["Feminino", "Masculino", "Feminino", "Masculino", None, "Feminino", "Masculino", "Feminino", "Masculino", "Feminino"],
    "Renda Anual (R$)": [45000, 60000, None, 80000, 50000, 70000, 100000, 30000, None, 85000],
    "Localização": ["São Paulo", "Rio de Janeiro", "São Paulo", "Belo Horizonte", "Salvador", "Porte Alegre", "São Paulo", "Curitiba", "Rio de Janeiro", "Belo Horizonte"],
    "Compras Anteriores": [3, 5, 2, None, 4, 6, 8, 1, 3, 7],
    "Resposta à Campanha": ["Sim", "Não", "Sim", "Não", "Sim", "Sim", "Não", "Não", "Sim", "Não"]
}
)


print(df)

   Cliente ID  Idade     Gênero  Renda Anual (R$)     Localização  \
0           1   25.0   Feminino           45000.0       São Paulo   
1           2   34.0  Masculino           60000.0  Rio de Janeiro   
2           3   28.0   Feminino               NaN       São Paulo   
3           4   45.0  Masculino           80000.0  Belo Horizonte   
4           5   39.0       None           50000.0        Salvador   
5           6    NaN   Feminino           70000.0    Porte Alegre   
6           7   50.0  Masculino          100000.0       São Paulo   
7           8   23.0   Feminino           30000.0        Curitiba   
8           9   29.0  Masculino               NaN  Rio de Janeiro   
9          10   40.0   Feminino           85000.0  Belo Horizonte   

   Compras Anteriores Resposta à Campanha  
0                 3.0                 Sim  
1                 5.0                 Não  
2                 2.0                 Sim  
3                 NaN                 Não  
4                 4.

Var: Idade
- Técnica: Imputação pela mediana da idade.
- Justificativa: A mediana é menos sensível a outliers e reflete a  
  tendência central dos dados.
- Impacto: Mantém a coerência da distribuição das idades e reduz viés causado por valores extremos.

In [ ]:
idade_imputer = SimpleImputer(strategy='median')
df['Idade'] = idade_imputer.fit_transform(df[['Idade']])

print(df['Idade'])

0    25.0
1    34.0
2    28.0
3    45.0
4    39.0
5    34.0
6    50.0
7    23.0
8    29.0
9    40.0
Name: Idade, dtype: float64


Var: 'Gênero', lidando com os valores faltantes.
- Técnica: Substituir por uma nova categoria chamada "Desconhecido".
- Justificativa: Não fazer suposições sobre o gênero por meior de media, moda ou regressao, irá evitar a introdução de viés nos dados, além de preservar informações sobre dados faltantes.
- Impacto: Garante que o modelo trate a falta de dados de forma explícita.
----

Var: 'Gênero', lidando com dados categoricos.
- Técnica: One-Hot Encoding.
-Justificativa: Representar cada categoria como uma coluna binária evita que os modelos interpretem categorias como ordinais quando utilizado o Label Encoding.
-Impacto: Garante que os modelos tratem cada gênero como igualmente importantes.

In [ ]:
df['Gênero'] = df['Gênero'].fillna('Desconhecido')

encoder = OneHotEncoder(sparse_output=False, drop='first')
genero_encoded = encoder.fit_transform(df[['Gênero']])

colunas_codificadas = encoder.get_feature_names_out(['Gênero'])
df_codificado = pd.DataFrame(genero_encoded, columns=colunas_codificadas)
df = pd.concat([df, df_codificado], axis=1)
df = df.drop(columns=['Gênero'])

print(df)



   Cliente ID  Idade  Renda Anual (R$)     Localização  Compras Anteriores  \
0           1   25.0           45000.0       São Paulo                 3.0   
1           2   34.0           60000.0  Rio de Janeiro                 5.0   
2           3   28.0               NaN       São Paulo                 2.0   
3           4   45.0           80000.0  Belo Horizonte                 NaN   
4           5   39.0           50000.0        Salvador                 4.0   
5           6   34.0           70000.0    Porte Alegre                 6.0   
6           7   50.0          100000.0       São Paulo                 8.0   
7           8   23.0           30000.0        Curitiba                 1.0   
8           9   29.0               NaN  Rio de Janeiro                 3.0   
9          10   40.0           85000.0  Belo Horizonte                 7.0   

  Resposta à Campanha  Gênero_Feminino  Gênero_Masculino  
0                 Sim              1.0               0.0  
1                 Não  

Var: Renda Anual, lidando com os valores faltantes.
- Técnica: Imputação pela média ou por regressão preditiva.
- Média: Têm vantagem por ser simples de aplicar. Outra alternativa é a imputação por regressão preditiva que garante maior precisão, mas a média pode ser usada caso o objetivo seja maior simplicidade.
- Impacto: Pode enviesar os dados principalmente por ser mais sensíveis a outlier's.

In [ ]:
media = df['Renda Anual (R$)'].mean()
df['Renda Anual (R$)'] = df['Renda Anual (R$)'].fillna(media)
print(df['Renda Anual (R$)'])

0     45000.0
1     60000.0
2     65000.0
3     80000.0
4     50000.0
5     70000.0
6    100000.0
7     30000.0
8     65000.0
9     85000.0
Name: Renda Anual (R$), dtype: float64


Var: Compras Anteriores, lidando com os valores faltantes
- Técnica: Imputação pela mediana.
- Justificativa: Dados discretos como "número de compras" frequentemente têm distribuição enviesada, tornando a mediana mais apropriada.
- Impacto: Preserva a consistência com o comportamento de compra observado.


In [ ]:
imputer = SimpleImputer(strategy='median')

df['Compras Anteriores'] = imputer.fit_transform(df[['Compras Anteriores']])

print(df)

   Cliente ID  Idade  Renda Anual (R$)     Localização  Compras Anteriores  \
0           1   25.0           45000.0       São Paulo                 3.0   
1           2   34.0           60000.0  Rio de Janeiro                 5.0   
2           3   28.0           65000.0       São Paulo                 2.0   
3           4   45.0           80000.0  Belo Horizonte                 4.0   
4           5   39.0           50000.0        Salvador                 4.0   
5           6   34.0           70000.0    Porte Alegre                 6.0   
6           7   50.0          100000.0       São Paulo                 8.0   
7           8   23.0           30000.0        Curitiba                 1.0   
8           9   29.0           65000.0  Rio de Janeiro                 3.0   
9          10   40.0           85000.0  Belo Horizonte                 7.0   

  Resposta à Campanha  Gênero_Feminino  Gênero_Masculino  
0                 Sim              1.0               0.0  
1                 Não  

Var: Resposta à Campanha, lidando com os valores
- Técnica: Label Encoding.
"Sim" → 1
"Não" → 0
- Justificativa: Como essa variável é o alvo do modelo preditivo; codificar como binária facilita a modelagem.
- Impacto: Facilita o treinamento do modelo, pois transforma a resposta em variável numérica.


In [ ]:
label_encoder = LabelEncoder()
df['Resposta à Campanha'] = label_encoder.fit_transform(df['Resposta à Campanha'])

print(df)

   Cliente ID  Idade  Renda Anual (R$)     Localização  Compras Anteriores  \
0           1   25.0           45000.0       São Paulo                 3.0   
1           2   34.0           60000.0  Rio de Janeiro                 5.0   
2           3   28.0           65000.0       São Paulo                 2.0   
3           4   45.0           80000.0  Belo Horizonte                 4.0   
4           5   39.0           50000.0        Salvador                 4.0   
5           6   34.0           70000.0    Porte Alegre                 6.0   
6           7   50.0          100000.0       São Paulo                 8.0   
7           8   23.0           30000.0        Curitiba                 1.0   
8           9   29.0           65000.0  Rio de Janeiro                 3.0   
9          10   40.0           85000.0  Belo Horizonte                 7.0   

   Resposta à Campanha  Gênero_Feminino  Gênero_Masculino  
0                    1              1.0               0.0  
1                    

Var: Localização, lidando com a variável categórica

- Técnica: One-Hot Encoding.
- Justificativa: Localização pode ter impacto significativo no comportamento de compra então utilizar o One-Hot Encoding, em vez de Label Encoding vai permitir tratar todas as regiões como iguais. Assim como, vai evitar a diminuição da variabilidade de dados em comparação com a imputação de valores de estatística descritiva.
- Impacto: Permite que o modelo capture as variações regionais sem introduzir hierarquia.

In [ ]:
encoder = OneHotEncoder(sparse_output=False, drop='first')
localizacao_encoded = encoder.fit_transform(df[['Localização']])

colunas_codificadas = encoder.get_feature_names_out(['Localização'])

df_codificado = pd.DataFrame(localizacao_encoded, columns=colunas_codificadas, index=df.index)
df = pd.concat([df.drop(columns=['Localização']), df_codificado], axis=1)

print(df)

   Cliente ID  Idade  Renda Anual (R$)  Compras Anteriores  \
0           1   25.0           45000.0                 3.0   
1           2   34.0           60000.0                 5.0   
2           3   28.0           65000.0                 2.0   
3           4   45.0           80000.0                 4.0   
4           5   39.0           50000.0                 4.0   
5           6   34.0           70000.0                 6.0   
6           7   50.0          100000.0                 8.0   
7           8   23.0           30000.0                 1.0   
8           9   29.0           65000.0                 3.0   
9          10   40.0           85000.0                 7.0   

   Resposta à Campanha  Gênero_Feminino  Gênero_Masculino  \
0                    1              1.0               0.0   
1                    0              0.0               1.0   
2                    1              1.0               0.0   
3                    0              0.0               1.0   
4           